# Reddit Clustering

For this project, we set out to explore the world of reddit, and illustrate the way users interact with the website.

## Research Question

We are interested in clustering subreddits, by using comment and submission data to discern connections between subreddits and the users who are active in them. We are hoping to see similar userbases in subreddits we did not expect, along with disconnects in userbases between subreddits that appear intuitively similar. We want to use a Clustering algorithm to connect subreddits in space, and then calculate the distance between subreddits.

## Data Sources

All of our data is coming from reddit. Reddit submission and comment data is publicly accessible, and reddit has a nice API structure. We are using the package PRAW (python reddit api wrapper), which makes the reddit api calls easier to use and python importable. We are attempting to build subreddit comment vectors for a large number of reddit users. We would like to create nested dictionaries, the first level key being a reddit user, the second level keys being a subreddit name, and the values being how many time that specific user posted to a specific subreddit. The below script generates this dictionary for us. We can then convert this dictionary into a csv and a dataframe that panda can use.

In [1]:
import praw
import numpy as np
import pylab as plt
%matplotlib inline
from scipy.stats import ttest_ind
import praw
import pandas as pd

reddit = praw.Reddit(client_id='tc_fFbWZrkDSRw',
                     client_secret='fTq7nFVzdkCHFZY7jWQvHmkLpwk',
                     user_agent='lhimelman',
                    username = 'lhimelman',
                    password = 'madisonave77')

# assume you have a Reddit instance bound to variable `reddit`
#top_level_comments = list(submission.comments)
commentFreq = {}
subredditList = ["funny","AskReddit","todayilearned","science","worldnews","pics"]
headers = []
# for subredditname in subredditList:
#     print("subreddit started")
subreddit = reddit.subreddit('funny')
#this is limited to started from a single subreddit and only looking at the top post, in our code we look at many more posts
for submission in subreddit.hot(limit=1):
    all_comments = submission.comments.list()
    for c in all_comments:
        try:
            print(c.author.name, ":")
            userCFreq = {}
            #this code looks at one comment per user, our actual code looks at all the users comments.
            for comment in reddit.redditor(c.author.name).comments.new(limit=1):
                # print(comment.subreddit)
                if comment.subreddit not in userCFreq:
                    userCFreq[comment.subreddit] = 1
                else:
                    userCFreq[comment.subreddit] += 1
                if comment.subreddit not in headers:
                    headers.append(comment.subreddit)

            commentFreq[c.author.name] = userCFreq
        except:
            pass

df = pd.DataFrame.from_dict(data=commentFreq, orient='index').fillna(0)
df.to_csv('redditDatafake.csv',header=headers)

etymologynerd :
__celli :
laskman :
draco123465 :
GhostOfWhatsIAName :
SwitchyMcSwitchface1 :
thejumbowumbo :
Mr-reader :


## Data Cleaning

The data came to us pretty clean. Reddit's api allows us to filter deleted comments and such. Our data cleaning plan is to make vectors out of the csv data that we can use for clustering later on. It is important that our vectors include zero for subreddits that a specific user did not comment on, and when our dataframe we took this into account
    
***Something we realized recently is that we need to filter porn. Didn't think that would be a big deal...but unfortunately it is.
    

In [2]:
import pandas as pd

df = pd.read_csv('data.csv',nrows=2000)
df

FileNotFoundError: File b'data.csv' does not exist

# Data Visualization

## Bar Graph

What are the most commented on subreddits from out sample?

In [ ]:
#from collections import Counter
    
new_data = pd.DataFrame(df, columns = ['funny', 'worldnews', 'AskReddit', 'askscience', 'vegan'])
#new_data

sums = {}
for c in new_data:
    sums[c] = new_data[c].sum()
sums

plt.bar(range(len(sums)), list(sums.values()), align = 'center')
plt.xticks(range(len(sums)), list(sums.keys()))

new_data.plot.bar()

## Scatter Plot

Number of subreddits commented on vs total number of posts for a user

In [ ]:
import scipy

df["total_posts"] = df.sum(axis=1)
df["total_posts"] = df["total_posts"].astype(int)


    
total_subreddits = []
for i in range(0, len(df.index)):
    subreddits = 0
    for j in range(1,len(df.columns)-1):
        if df.iat[i,j] != 0.0:
            subreddits += 1
    total_subreddits.append(subreddits)
  
df["total_subreddits"] = total_subreddits       
            

plt.scatter(df.total_posts,df.total_subreddits, color='k', s=20, marker="^")
plt.xlabel('number of posts')
plt.ylabel('number of subreddits')
plt.show()


## Line graph

Is reddit user posting a normal distribution?

In [ ]:
df["total_posts"].plot(kind="density")
scipy.stats.normaltest(df["total_posts"])

The number of posts reddit users make is not nearly a normal distribution. There are lots of people with almost no posts, and lots of users with tons of posts, and a large drop inbetween.There is no consistent distribution of how may posts a reddit user makes.